In [1]:
import numpy as np
import math
import random

In [2]:
def generate_binary_array(num_centroids):
    # Determine k from num_centroids
    k = int(np.log2(num_centroids))

    # Generate all possible binary combinations of length k
    b = np.array([list(map(int, np.binary_repr(i, width=k))) for i in range(num_centroids)])

    return b

In [3]:
def channel_with_memory(num_level, epsilon, delta):
    Pr = np.zeros((num_level, num_level))
    n = int(np.log2(num_level))

    # Transition probability matrix for the binary symmetric channel with memory
    Pr_z = np.array([
        [(1 - epsilon + delta) / (1 + delta), epsilon / (1 + delta)],
        [(1 - epsilon) / (1 + delta), (epsilon + delta) / (1 + delta)]
    ])

    for x in range(num_level):
        for y in range(num_level):
            binary_x = np.array([int(bit) for bit in np.binary_repr(x, width=n)])
            binary_y = np.array([int(bit) for bit in np.binary_repr(y, width=n)])
            binary_z = binary_x ^ binary_y  # XOR operation

            if binary_z[0] == 1:
                probability = epsilon
            else:
                probability = 1 - epsilon
            for i in range(1, n):
                probability *= Pr_z[binary_z[i - 1], binary_z[i]]

            Pr[x, y] = probability

    return Pr

In [4]:
# Simulated Annealing Algorithm
def simulated_annealing(T_0, alpha, T_f, b, N_fail, N_success, N_cut, k, epsilon, delta, centroids, partitions, num_centroids):
    T = T_0
    count = 0
    count_success = 0
    count_fail = 0
    b_history = []

    prob_points = []

    # Loop over each partition
    for partition in partitions:
        # Calculate the probability of samples falling in this partition
        prob = len(partition) / 500000
        prob_points.append(prob)

    conditional_prob = channel_with_memory(num_centroids, epsilon, delta)

    while T > T_f and count_fail < N_fail:

        b_prime = random.sample(b, len(b))

        delta_Dc = 0

        distortion_b = 0
        distortion_b_prime = 0

        for g in range(0, num_centroids):
            for h in range(0, num_centroids):
                distortion_b = distortion_b + prob_points[g] * conditional_prob[h,b[g]] * ((centroids[b[g]]-centroids[h])**2)
                distortion_b_prime = distortion_b_prime + prob_points[g] * conditional_prob[h, b_prime[g]] * ((centroids[b_prime[g]]-centroids[h])**2)

        distortion_b = distortion_b * (1 / k)
        distortion_b_prime = distortion_b_prime * (1 / k)
        delta_Dc = distortion_b_prime - distortion_b
        b_history.append(distortion_b)

        if delta_Dc <= 0:
            b = b_prime
            count_success = count_success + 1
            count_fail = 0
        else:
            rand_num = random.uniform(0, 1)
            if rand_num <= math.exp(-delta_Dc / T):
                b = b_prime
            count_fail = count_fail + 1

        if count >= N_cut or count_success >= N_success:
            T = alpha * T
            count = 0
            count_success = 0
        count = count + 1

    return b

In [5]:
# Function to generate normalized source signal that will be used for training
def generate_source_signal(distribution, num_samples=500000):

    if distribution.lower() == 'laplace':
        source = np.random.laplace(loc=0, scale=np.sqrt(1/2), size=num_samples)
    else:
        source = np.random.normal(loc=0, scale=1, size=num_samples)

    # Normalize (zero-mean, unit variance)
    source = (source - np.mean(source)) / np.std(source)

    return source

In [6]:
def generate_initial_codebook(source, num_centroids):
    min_samples = np.min(source)
    max_samples = np.max(source)
    width = (max_samples - min_samples) / num_centroids
    centroids = []
    for i in range(num_centroids):
        # Calculate the current centroid
        centroid_current = min_samples + (i + 0.5) * width
        centroids.append(centroid_current)

    return centroids

In [7]:
def cosq_design(source, current_codebook, epsilon, b_obtained, tol=1e-4, max_iter=100):
    n_codewords = len(current_codebook)
    P_Y_given_X = channel_with_memory(n_codewords, epsilon, 10)

    print(P_Y_given_X)

    # Initialize codebook (ENSURE IT'S A NUMPY ARRAY)
    codebook = np.asarray(current_codebook.copy())  # Convert to NumPy array

    signal_power = np.mean(source ** 2)

    for iteration in range(max_iter):
        # --------------------------------------------------
        # Generalized NNC (Nearest Neighbor Condition)
        # --------------------------------------------------

        # Initialize the partitions
        partitions = [[] for _ in range(n_codewords)]

        for v in source:
          distortions = []

          # Iterate over each partition index
          for i in range(num_centroids):
            # Assuming v is assigned to i
            distortion = 0

            for j in range(num_centroids):
              # Compute the total distortion assuming that v is assigned to partition[i]
              distortion += P_Y_given_X[j, b_obtained[i]] * ((v - codebook[j])**2)

            distortions.append(distortion)

          # To find the minimum i from the distortion list
          min_distortion_idx = np.argmin(distortions)
          # Add v to that partition
          partitions[min_distortion_idx].append(v)

        # To visualize the partition of the line
        print(len(partitions[0]),len(partitions[1]),len(partitions[2]),len(partitions[3]))
        # --------------------------------------------------
        # Generalized CC (Centroid Condition)
        # --------------------------------------------------
        new_codebook = np.zeros_like(codebook)
        for i in range(n_codewords):
            numerator = 0.0
            denominator = 0.0

            for j in range(n_codewords):
                # If there is no element in that partition, skip that partition set
                if len(partitions[j]) == 0:
                    continue


                prob = P_Y_given_X[i, b_obtained[j]]
                sum_v = np.sum(partitions[j])
                count = len(partitions[j])

                numerator += prob * sum_v
                denominator += prob * count

            new_codebook[i] = numerator / denominator




        print(new_codebook)
        # Check convergence
        codebook_change = np.max(np.abs(new_codebook - codebook))
        codebook = new_codebook.copy()

        if codebook_change < tol:
            break


    # Calculate MSE
    summation = 0
    for i in range(n_codewords):
        for x in partitions[i]:
            for j in range(n_codewords):
                summation = summation + P_Y_given_X[b_obtained[i], j]*((x - codebook[j])**2)
    
    summation = summation / len(sampled_source)
    
    snr = 10 * np.log10(signal_power / summation)

    return codebook, partitions, snr

In [8]:
#--------------------------------------------------------------
# Parameter declarations
#--------------------------------------------------------------

num_centroids = 4
# For SA
T_0 = 10
alpha = 0.97
T_f = 0.00025
N_fail = 50000
N_success = 5
N_cut = 200
k = 10
# For channel
delta = 10

initial_distribution = 'laplace'

sampled_source = generate_source_signal(initial_distribution)

codebooks_set = []

# Create an empty array b with length num_centroids
init_b = [0] * num_centroids

# Populate the array with values from 0 to num_centroids - 1
for i in range(num_centroids):
    init_b[i] = i


In [9]:
# COSQ for noiseless. Works for a preset low noise epsilon
init_codebook = generate_initial_codebook(sampled_source, num_centroids)
codebooks_set.append(np.array(init_codebook))

print(init_codebook)
noiseless_codebook, noiseless_partition, noiseless_snr = cosq_design(sampled_source, codebooks_set[-1],  1e-11, init_b)
print(noiseless_snr)
codebooks_set.append(np.array(noiseless_codebook))

[-6.727348245762521, -2.3262130897619446, 2.0749220662386296, 6.476057222239206]
[[1.00000000e+00 9.09090909e-13 9.09090909e-13 9.09090909e-12]
 [9.09090909e-13 1.00000000e+00 9.09090909e-12 9.09090909e-13]
 [9.09090909e-13 9.09090909e-12 1.00000000e+00 9.09090909e-13]
 [9.09090909e-12 9.09090909e-13 9.09090909e-13 1.00000000e+00]]
399 208721 290258 622
[-5.23395709 -0.82452059  0.58931247  5.03288154]
3480 208080 283774 4666
[-3.72513823 -0.77622221  0.55675655  3.53338304]
10292 203736 272103 13869
[-2.96012573 -0.70824397  0.50189376  2.75388357]
18627 197332 259070 24971
[-2.54048545 -0.64685026  0.45012863  2.33675275]
26225 191261 247760 34754
[-2.29916438 -0.60035324  0.41193852  2.10214247]
32211 186570 239015 42204
[-2.1542421  -0.56737014  0.38630169  1.96456832]
36553 183369 232755 47323
[-2.06519956 -0.54458873  0.37042179  1.8834902 ]
39635 181373 228325 50667
[-2.00824503 -0.52846659  0.3611783   1.83512593]
41715 180431 225057 52797
[-1.97227447 -0.51692917  0.35632562  

In [10]:
# Once we obtain the noiseless_codebook, we perform SA algorithm to obtain the shuffling before we train with noisy channels
b_from_sa = simulated_annealing(T_0, alpha, T_f, init_b, N_fail, N_success, N_cut, k, 0.005, delta, noiseless_codebook, noiseless_partition, num_centroids)

print(b_from_sa)

[3, 1, 2, 0]


In [11]:
# Starting with noiseless codebook and b obtained from SA algorithm, run the training for epsilon = 0.005
codebook_1, partition_1, snr_1 = cosq_design(sampled_source, codebooks_set[-1], 0.005, b_from_sa)
print(snr_1)
codebooks_set.append(np.array(codebook_1))

[[9.94547727e-01 4.52272727e-04 4.52272727e-04 4.54772727e-03]
 [4.52272727e-04 9.94547727e-01 4.54772727e-03 4.52272727e-04]
 [4.52272727e-04 4.54772727e-03 9.94547727e-01 4.52272727e-04]
 [4.54772727e-03 4.52272727e-04 4.52272727e-04 9.94547727e-01]]
46718 212486 186935 53861
[-1.77189097 -0.37199342  0.46572314  1.86613799]
47338 211928 186365 54369
[-1.76538286 -0.37040941  0.46326763  1.85710428]
47705 211522 186141 54632
[-1.76202395 -0.3698228   0.46156967  1.8518205 ]
47946 211191 186096 54767
[-1.76029365 -0.36976325  0.46023213  1.84837972]
48109 210916 186130 54845
[-1.75929047 -0.36989317  0.459187    1.84606531]
48225 210699 186202 54874
[-1.75890584 -0.37016707  0.45833822  1.84442959]
48324 210514 186280 54882
[-1.75878519 -0.37048888  0.45757769  1.84304091]
48408 210349 186369 54874
[-1.75886603 -0.37085575  0.45688241  1.84186863]
48478 210191 186470 54861
[-1.75901188 -0.37123507  0.45624439  1.84089503]
48535 210044 186582 54839
[-1.75927212 -0.3716402   0.45565967 

In [12]:
codebook_2, partition_2, snr_2 = cosq_design(sampled_source, codebooks_set[-1], 0.01, b_from_sa)
print(snr_2)
codebooks_set.append(np.array(codebook_2))

[[9.891e-01 9.000e-04 9.000e-04 9.100e-03]
 [9.000e-04 9.891e-01 9.100e-03 9.000e-04]
 [9.000e-04 9.100e-03 9.891e-01 9.000e-04]
 [9.100e-03 9.000e-04 9.000e-04 9.891e-01]]
48242 209277 188595 53886
[-1.75045525 -0.37467548  0.44859451  1.81922267]
48875 208662 188073 54390
[-1.74410367 -0.37323084  0.44603246  1.81056133]
49225 208215 187907 54653
[-1.74080054 -0.37272499  0.44436181  1.80582034]
49459 207911 187846 54784
[-1.73913466 -0.37263436  0.44313911  1.80268835]
49620 207666 187854 54860
[-1.73815906 -0.37271588  0.44219227  1.80054845]
49752 207466 187889 54893
[-1.73770931 -0.3729357   0.44135468  1.79881317]
49841 207291 187966 54902
[-1.73756403 -0.37323137  0.44067645  1.79765201]
49913 207119 188071 54897
[-1.73759277 -0.37359421  0.44003701  1.79671969]
49968 206952 188198 54882
[-1.73774788 -0.37399934  0.43944915  1.79601369]
50017 206801 188321 54861
[-1.73797709 -0.37440508  0.43890452  1.79538884]
50062 206675 188428 54835
[-1.73826766 -0.37478886  0.43841687  1.7

In [13]:
codebook_3, partition_3, snr_3 = cosq_design(sampled_source, codebooks_set[-1], 0.05, b_from_sa)
print(snr_3)
codebooks_set.append(np.array(codebook_3))


[[0.94568182 0.00431818 0.00431818 0.04568182]
 [0.00431818 0.94568182 0.04568182 0.00431818]
 [0.00431818 0.04568182 0.94568182 0.00431818]
 [0.04568182 0.00431818 0.00431818 0.94568182]]
44965 210825 195510 48700
[-1.64155666 -0.36922244  0.41764461  1.66668574]
49860 205585 191445 53110
[-1.59178632 -0.35671597  0.39932563  1.61249347]
52775 201993 189537 55695
[-1.56411897 -0.35052175  0.38791715  1.58211939]
54510 199642 188691 57157
[-1.5487974  -0.34764718  0.38074325  1.56478257]
55580 198046 188410 57964
[-1.5402873  -0.34660837  0.37592775  1.55447792]
56233 196933 188455 58379
[-1.53576895 -0.34659581  0.37261623  1.54840157]
56673 196178 188567 58582
[-1.53335488 -0.34697695  0.37023212  1.54447599]
56937 195624 188787 58652
[-1.53232429 -0.34763088  0.36852909  1.5422244 ]
57108 195191 189038 58663
[-1.53193154 -0.34834868  0.36723294  1.5408308 ]
57243 194871 189232 58654
[-1.5317626  -0.34896704  0.36621289  1.53976427]
57338 194591 189440 58631
[-1.53177645 -0.34957662 

In [14]:
codebook_4, partition_4, snr_4 = cosq_design(sampled_source, codebooks_set[-1], 0.1, b_from_sa)
print(snr_4)
codebooks_set.append(np.array(codebook_4))

[[0.89181818 0.00818182 0.00818182 0.09181818]
 [0.00818182 0.89181818 0.09181818 0.00818182]
 [0.00818182 0.09181818 0.89181818 0.00818182]
 [0.09181818 0.00818182 0.00818182 0.89181818]]
49116 201786 198944 50154
[-1.40972224 -0.33822274  0.34803479  1.40989598]
55452 195622 192608 56318
[-1.35778568 -0.32078742  0.32970005  1.35804494]
58971 192061 189239 59729
[-1.33058547 -0.31176603  0.31974087  1.33085416]
60729 190040 187615 61616
[-1.31658047 -0.30684467  0.31490753  1.31708435]
61687 189063 186643 62607
[-1.30923272 -0.30419889  0.312403    1.30979942]
62263 188524 186048 63165
[-1.30502331 -0.30272462  0.31089832  1.3055712 ]
62569 188225 185746 63460
[-1.30280496 -0.30196708  0.31008186  1.30334084]
62736 188054 185584 63626
[-1.30157472 -0.30154236  0.30963626  1.30211208]
62834 187961 185489 63716
[-1.3008871  -0.30131698  0.30937062  1.30141383]
62888 187908 185443 63761
[-1.30052911 -0.30121577  0.30921321  1.30104281]
62922 187884 185418 63776
[-1.30036347 -0.30119957 

In [15]:
codebook_5, partition_5, snr_5 = cosq_design(sampled_source, codebooks_set[-1], 0.05, b_from_sa)
print(snr_5)
codebooks_set.append(np.array(codebook_5))

[[0.94568182 0.00431818 0.00431818 0.04568182]
 [0.00431818 0.94568182 0.04568182 0.00431818]
 [0.00431818 0.04568182 0.94568182 0.00431818]
 [0.04568182 0.00431818 0.00431818 0.94568182]]
72078 178930 176291 72701
[-1.4060424  -0.31237329  0.32004799  1.40855229]
65749 185150 182634 66467
[-1.45859947 -0.32956592  0.33766532  1.46184503]
62172 188660 186150 63018
[-1.48961832 -0.33931525  0.34814016  1.49382666]
60220 190737 188005 61038
[-1.50792769 -0.34496075  0.35405739  1.51211498]
59182 191914 188999 59905
[-1.5184943  -0.34827551  0.35721286  1.52216358]
58582 192516 189624 59278
[-1.52445673 -0.35019804  0.35897425  1.52798735]
58209 192862 190021 58908
[-1.52803357 -0.35134692  0.36006525  1.53159882]
57996 193080 190242 58682
[-1.53020161 -0.35204363  0.36069502  1.53369612]
57880 193170 190368 58582
[-1.53120162 -0.35235668  0.36103582  1.53479998]
57802 193255 190440 58503
[-1.53196823 -0.35258505  0.36128318  1.53556433]
57753 193297 190487 58463
[-1.53237373 -0.35269961 

In [16]:
codebook_6, partition_6, snr_6 = cosq_design(sampled_source, codebooks_set[-1], 0.01, b_from_sa)
print(snr_6)
codebooks_set.append(np.array(codebook_6))

[[9.891e-01 9.000e-04 9.000e-04 9.100e-03]
 [9.000e-04 9.891e-01 9.100e-03 9.000e-04]
 [9.000e-04 9.100e-03 9.891e-01 9.000e-04]
 [9.100e-03 9.000e-04 9.000e-04 9.891e-01]]
63799 187186 184405 64610
[-1.62560314 -0.36260056  0.37250057  1.63273306]
59143 191991 188904 59962
[-1.67619853 -0.37693134  0.38760792  1.68412966]
56317 194835 191594 57254
[-1.70750074 -0.38544562  0.39717387  1.71718068]
54594 196617 193142 55647
[-1.72677247 -0.39040679  0.403308    1.73812014]
53529 197804 193984 54683
[-1.73860551 -0.3931819   0.40737868  1.75138108]
52872 198512 194430 54186
[-1.74482729 -0.39442636  0.41006631  1.75964044]
52444 199034 194621 53901
[-1.74843973 -0.39491006  0.41203074  1.76505743]
52163 199462 194637 53738
[-1.75053005 -0.39492822  0.41354908  1.76862725]
51973 199839 194550 53638
[-1.75182361 -0.39471142  0.41477882  1.77104616]
51831 200114 194456 53599
[-1.75235848 -0.39436886  0.41576095  1.77284356]
51719 200328 194362 53591
[-1.7525044  -0.39398098  0.41657869  1.7

In [17]:
codebook_7, partition_7, snr_7 = cosq_design(sampled_source, codebooks_set[-1], 0.005, b_from_sa)
print(snr_7)
codebooks_set.append(np.array(codebook_7))

[[9.94547727e-01 4.52272727e-04 4.52272727e-04 4.54772727e-03]
 [4.52272727e-04 9.94547727e-01 4.54772727e-03 4.52272727e-04]
 [4.52272727e-04 4.54772727e-03 9.94547727e-01 4.52272727e-04]
 [4.54772727e-03 4.52272727e-04 4.52272727e-04 9.94547727e-01]]
51337 203609 189972 55082
[-1.75566547 -0.38309302  0.43203523  1.80201987]
50731 204231 190490 54548
[-1.76240962 -0.38470015  0.43438695  1.81019565]
50333 204630 190762 54275
[-1.76589906 -0.38536197  0.4360867   1.8156002 ]
50094 204981 190808 54117
[-1.76792721 -0.38550738  0.43736137  1.81886489]
49935 205267 190748 54050
[-1.76880165 -0.38529513  0.43840648  1.82103545]
49815 205517 190645 54023
[-1.76916659 -0.38494055  0.43932086  1.82267267]
49722 205717 190540 54021
[-1.76921215 -0.3845517   0.44008504  1.82393954]
49625 205908 190437 54030
[-1.76912061 -0.38412165  0.4408693   1.825261  ]
49552 206086 190308 54054
[-1.76883574 -0.38363771  0.44157129  1.82625325]
49490 206285 190151 54074
[-1.76859867 -0.3831343   0.44227014 